In [86]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import xgboost as xgb
from glob import glob

import os
from pathlib import Path

# Styling
color_pal = sns.color_palette("Set2")
plt.style.use('seaborn-v0_8-colorblind') 

# Warnings control
import warnings 
warnings.filterwarnings('ignore')

In [87]:
def get_stock_delta_cols(df):
    df["Delta"] = df["Close"] - df["Close"].shift(1)
    df["Delta %"] = df["Delta"] / df["Close"]
    return df


In [88]:
def get_testing_correlations():
    base = pd.DataFrame()

    for f in glob("stock-price-predictions/price/*"):
        company = f.split("/")[-1][:-4]

        data_path_stock = Path(f'stock-price-predictions/price/{company}.csv')
        df = pd.read_csv(data_path_stock, header=0)

        df = df[["Date", "Close"]].rename(columns={"Close":company})

        if base.empty:
            base = df
        else:
            base = base.merge(
                right=df, 
                how="left",
                on="Date"
            )

    base.set_index('Date', inplace=True)
    base.index = pd.to_datetime(base.index)
    base = base.loc[base.index < '01-01-2016'] #restricts to train domain

    return base.corr(numeric_only=False)

In [89]:
df = get_testing_correlations()

In [90]:
df

,XOM,MSFT,CELG,MMM,C,BCH,CHL,MO,AEP,BA,...,RDS-B,EXC,PG,PCLN,SLB,GD,PCG,WMT,SNY,D
XOM,1.000000,-0.190075,-0.363670,-0.087806,-0.170885,0.394913,-0.474108,-0.472340,-0.163960,-0.103567,...,0.931245,0.303763,0.375033,-0.026506,0.671153,-0.261683,-0.410485,0.544238,0.381341,0.030343
MSFT,-0.190075,1.000000,0.878255,0.895167,0.729967,-0.866535,0.452204,0.889700,0.848522,0.836716,...,-0.314167,0.068309,0.487353,0.839533,0.348328,0.936744,0.751564,-0.103213,0.037858,0.825519
CELG,-0.363670,0.878255,1.000000,0.936550,0.823256,-0.839450,0.619676,0.927439,0.866920,0.892143,...,-0.451101,0.043373,0.510561,0.809457,0.238922,0.954460,0.821465,0.020014,0.091065,0.858904
MMM,-0.087806,0.895167,0.936550,1.000000,0.789112,-0.797346,0.514899,0.858215,0.888530,0.933980,...,-0.208316,0.152981,0.656540,0.853873,0.452553,0.947541,0.774893,0.182016,0.138254,0.928977
C,-0.170885,0.729967,0.823256,0.789112,1.000000,-0.558425,0.372536,0.655723,0.613567,0.829349,...,-0.306855,-0.105248,0.617849,0.759793,0.327142,0.743924,0.544933,0.073142,0.410608,0.716639
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GD,-0.261683,0.936744,0.954460,0.947541,0.743924,-0.896845,0.562651,0.926854,0.900211,0.869605,...,-0.334607,0.143594,0.470351,0.840540,0.340681,1.000000,0.811519,-0.020805,0.052905,0.883964
PCG,-0.410485,0.751564,0.821465,0.774893,0.544933,-0.772472,0.673439,0.900142,0.906379,0.619861,...,-0.500779,0.335391,0.398036,0.486036,-0.022187,0.811519,1.000000,0.056589,-0.120374,0.699085
WMT,0.544238,-0.103213,0.020014,0.182016,0.073142,0.171154,0.069140,-0.151676,0.158220,0.095940,...,0.512203,0.561510,0.664136,-0.070386,0.360615,-0.020805,0.056589,1.000000,0.253073,0.265298
SNY,0.381341,0.037858,0.091065,0.138254,0.410608,0.129545,-0.137088,-0.098448,0.031460,0.259400,...,0.405866,0.089570,0.322142,0.289465,0.540623,0.052905,-0.120374,0.253073,1.000000,0.243464
